In [ ]:
#' ' means CPU whereas '/device:G:0' means GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 11.5 GB  | Proc size: 1.3 GB
GPU RAM Free: 14799MB | Used: 310MB | Util   2% | Total 15360MB


In [ ]:
# !kill -9 -1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
cd 'drive/MyDrive/AgriWise/ML/plant diseases'

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-44-c58fb5024c45>", line 1, in <module>
    get_ipython().run_line_magic('cd', "'drive/MyDrive/AgriWise/ML/plant diseases'")
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2314, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/magics/osm.py", line 334, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core

OSError: ignored

In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')

# for gpu in gpus:
#   tf.experimental.set_memory_growth(gpu, True)

# Preprocessing 

In [ ]:
batch_size = 7

In [ ]:
root_dir = "/content/drive/MyDrive/AgriWise/ML/plant diseases/fiveClasses/"

train_set = tf.keras.utils.image_dataset_from_directory(
    root_dir + "train", 
    labels='inferred',
    label_mode = 'categorical',
    batch_size=batch_size,
    image_size=(256, 256),
    seed = 42)

val_set = tf.keras.utils.image_dataset_from_directory(
    root_dir + "val", 
    labels='inferred',
    label_mode = 'categorical',
    batch_size=batch_size,
    image_size=(256, 256),
    seed = 42)

test_set = tf.keras.utils.image_dataset_from_directory(
    root_dir + "test", 
    labels='inferred',
    label_mode = 'categorical',
    batch_size=batch_size,
    image_size=(256, 256),
    seed = 42)

Found 37077 files belonging to 25 classes.
Found 9282 files belonging to 25 classes.
Found 11588 files belonging to 25 classes.


In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./256)
normalized_training_set = train_set.map(lambda x, y: (normalization_layer(x), y))
training_image_batch, training_labels_batch = next(iter(normalized_training_set))

In [ ]:
normalized_val_set = val_set.map(lambda x, y: (normalization_layer(x), y))
val_image_batch, val_labels_batch = next(iter(normalized_val_set))

# VGG Model

In [ ]:
# keep include_top false to retrain it with new classes 
pre_trained_model = tf.keras.applications.VGG16(input_shape=(256, 256, 3), include_top=False, weights="imagenet", classifier_activation="softmax")

In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

input_3
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activity_regularizer = regularizers.L2 (1e-5), activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(25, activity_regularizer = regularizers.L2 (1e-5), activation='softmax')(x)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-ca63462ee3c8>", line 4, in <module>
    x = tf.keras.layers.Dense(512, activity_regularizer = regularizers.L2 (1e-5), activation='relu')(x)
NameError: name 'regularizers' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context,

NameError: ignored

In [ ]:
with tf.device(device_name):
    model = tf.keras.Model(pre_trained_model.input, x)      # Create a TensorFlow model.
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc']) # Compile the model on the GPU.
    model.summary()

In [ ]:
vgg_clf = model.fit(normalized_training_set,
epochs = 50,
validation_data=normalized_val_set,
batch_size = batch_size,
verbose = 1)

## Evaluate the model

In [ ]:
result = model.evaluate(test_set, batch_size=batch_size)
print("test_loss, test accuracy",result)

## Save the model


In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/Image Classification/VGG_Skin_Classifier.json", "w") as json_file:
  json_file.write(model_json)
  model.save("/content/drive/MyDrive/Image Classification/VGG_Skin_Classifier.h5")
  print("Saved model to disk")
  model.save_weights("/content/drive/MyDrive/Image Classification/VGG_Skin.h5")